In [1]:
pip install cdsapi

  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=fc4322bbe0568b8123770137b34e1cea362d8d7ca5304594bd14081abb1a00e6
  Stored in directory: /root/.cache/pip/wheels/1d/2e/3c/3746e0cd076320584d6f47b688da7b13a5d5d1a92606779ca4
Successfully built cdsapi


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
url = 'url:https://cds.climate.copernicus.eu/api/v2'
key = 'key:62276:be0ea6fa-ec28-46ea-9517-c6d5acd74af4'
with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

with open('/root/.cdsapirc') as f:
    print(f.read())

url:https://cds.climate.copernicus.eu/api/v2
key:62276:be0ea6fa-ec28-46ea-9517-c6d5acd74af4


In [4]:
pip install global_land_mask

     |████████████████████████████████| 1.8 MB 14.5 MB/s 


In [5]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle as vc 
from datetime import datetime
#import xlsxwriter 
import csv
from datetime import timedelta
import math
import os
import datetime
import cdsapi
from global_land_mask import globe
BASIN = 'WPSep2020ToSep2021' # set WP, SP, EP, NI, SI
path = '/content/drive/MyDrive/IJCAICodeToSubmit' #Change it accordingly. 
path1 = path + '/'+BASIN+'/'

data = pd.read_csv(path + '/ibtracs.since1980.list.v04r00.csv')
data1 = data[data['BASIN'] == 'WP']
columns = ['SID', 'ISO_TIME','USA_SSHS','LAT','LON'] #Extracting lat, lon, sid and time and cateogry .
data2 = data1[columns]
data2 = data2[data2['ISO_TIME'] > '2020-09-00 00:00:00']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,8,9,14,19,20,25,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
if not os.path.exists(path1):
    os.makedirs(path1)

def getValueInt(value):
  try:
    int(value)
    return int(value)
  except ValueError:
    return None

def getValueFloat(value):
  try:
    float(value)
    return float(value)
  except ValueError:
    return None

sidIndex = data2.columns.get_loc('SID')
timeIndex = data2.columns.get_loc('ISO_TIME')
catIndex = data2.columns.get_loc('USA_SSHS')
latIndex = data2.columns.get_loc('LAT')
lonIndex = data2.columns.get_loc('LON')

sids = pd.unique(data2['SID'])

alldata = []
for sid in sids:
  dataa = []
  data3 = data2[data2['SID'] == sid]
  values = data3.values
  length = len(values)
  val = values[0]
  startTime = val[timeIndex]
  endTime = values[-1][timeIndex]
  lat = getValueFloat(val[latIndex])
  long = getValueFloat(val[lonIndex])
  if long > 180:
    long = (long - 360)
  if long < -180:
    long = (long + 360)

  cateogry = -10000
  for val in values:
    cat = getValueInt(val[catIndex])
    if cat!=None:
      if cat > cateogry:
        cateogry = cat

  # Cateogry assigned to TC in IBTrACS as it progress
  # -5 = Unknown [XX], -4 = Post-tropical [EX, ET, PT], -3 = Miscellaneous disturbances [WV, LO, DB, DS, IN, MD], -2 = Subtropical [SS, SD]
  #-1 = Tropical depression (W<34), 0 = Tropical storm [34<W<64], 1 = Category 1 [64<=W<83], 2 = Category 2 [83<=W<96], 3 = Category 3 [96<=W<113], 4 = Category 4 [113<=W<137], 5 = Category 5 [W >= 137]
  if cateogry < -1:
    continue
  cateogry  = 1 # Assigning class label 1 to all TCs

  dataa.append(sid)
  dataa.append(startTime)
  #dataa.append(endTime)
  dataa.append(lat)
  dataa.append(long)
  dataa.append(cateogry)
  alldata.append(dataa)

dataa = pd.DataFrame(alldata, columns = ['SID', 'startTime','lat','lon','cateogry'])

dataa.to_csv(path1+BASIN+'OceanCenter1.csv') 

In [ ]:
from random import randrange
from datetime import timedelta
from global_land_mask import globe
from sklearn.neighbors import KernelDensity
import random

data2 = pd.read_csv(path1+BASIN+'OceanCenter1.csv', index_col=0)
totalNumber = data2.shape[0] 
print("totalNumber", totalNumber)
#Setting number of class 0, equals to class 1
noZeroClass = int(totalNumber*1.0)
print("noZeroClass  ", noZeroClass)

#Generating a kernel density function on the locations to generte random locations
positions = data2[['lat','lon']].values
posKde = KernelDensity(kernel='gaussian')
posKde.fit(positions)

sidIndex = data2.columns.get_loc('SID')
stimeIndex = data2.columns.get_loc('startTime')
catIndex = data2.columns.get_loc('cateogry')
latIndex = data2.columns.get_loc('lat')
lonIndex = data2.columns.get_loc('lon')

startDate =  datetime.datetime.strptime('1980-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
endDate = datetime.datetime.strptime('2021-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')

values = data2.values
startDates = values[:,stimeIndex]
startLocations = values[:,[latIndex,lonIndex]]

windowDays = 5
windowDegree = 5
secondsIn5Days = windowDays*24*60*60

#Function to check if random date liest in a window of five days
def isWithin5Days(date1, date2):
  date11 = datetime.datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
  #date22 = datetime.datetime.strptime(date2, '%Y-%m-%d %H:%M:%S')
  delta = date2 - date11
  days = abs(delta.days)
  secs = days * 24 * 60 * 60
  if  secs <= secondsIn5Days:
    return True
  return False
    
# Function to check if random generated location lies within 5 degrees. 
def isWithin5Degree(lat1, lon1, lat2, lon2):
  if abs(lat1 - lat2) <= windowDegree and abs(lon1 - lon2) <= windowDegree:
    return True
  return False

def isValid(date2, lat2, lon2):
  length = len(startDates)
  for i in range(length):
    date1 = startDates[i]
    if isWithin5Days(date1, date2):
      print("Within 5 days", date1, "   ", date2)
      locs = startLocations[i]
      lat1 = locs[0]
      lon1 = locs[1]
      if isWithin5Degree(lat1, lon1, lat2, lon2):
        print("Within 5 degree", lat1, "  ",lon1,"  ", lat2, "  ", lon2)
        return False

  return True


def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def randomLocation():
    return posKde.sample()[0]

alldata = []
count = 0
while count < noZeroClass:
  data = []
  print("++++++++++++++++")
  rDate = random_date(startDate, endDate)
  print("rDate   ",rDate)
  onLand = True
  while onLand:
    lat, lon = randomLocation()
    if lon > 180:
      lon = (lon - 360)
    if lon < -180:
      lon = (lon + 360)
    print("lat  ", lat, "lon ", lon)
    onLand = globe.is_land(lat, lon)
  if isValid(rDate, lat, lon):
    print("Valid pair ",rDate, "   " ,lat, "   ", lon )
    sid = str(rDate.year)+"RanCat0NI"+str(count)
    date = rDate.strftime('%Y-%m-%d %H:%M:%S')
    print("sid ", sid)
    print("date  ", date)
    data.append(sid)
    data.append(date)
    data.append(lat)
    data.append(lon)
    data.append(0)
    alldata.append(data)
    count = count + 1
  else:
    print("Not valid ", rDate, "   " ,lat, "   ", lon)

data3 = pd.DataFrame(alldata, columns = ['SID', 'startTime','lat','lon','cateogry'])

data4 = pd.concat([data2, data3])
data4.to_csv(path1+BASIN+'OceanCenterCombined.csv') 

totalNumber 25
noZeroClass   25
++++++++++++++++
rDate    2014-10-14 09:04:54
lat   20.73915986089647 lon  144.54839224388172
Valid pair  2014-10-14 09:04:54     20.73915986089647     144.54839224388172
sid  2014RanCat0NI0
date   2014-10-14 09:04:54
++++++++++++++++
rDate    1988-07-10 17:05:54
lat   27.71981436615318 lon  127.0027014964634
Valid pair  1988-07-10 17:05:54     27.71981436615318     127.0027014964634
sid  1988RanCat0NI1
date   1988-07-10 17:05:54
++++++++++++++++
rDate    1993-06-03 05:41:41
lat   13.094483493365995 lon  128.59109924764678
Valid pair  1993-06-03 05:41:41     13.094483493365995     128.59109924764678
sid  1993RanCat0NI2
date   1993-06-03 05:41:41
++++++++++++++++
rDate    2014-09-06 06:51:19
lat   11.742739115747659 lon  131.05706504212586
Valid pair  2014-09-06 06:51:19     11.742739115747659     131.05706504212586
sid  2014RanCat0NI3
date   2014-09-06 06:51:19
++++++++++++++++
rDate    2021-03-06 20:36:21
lat   7.996971724384779 lon  138.47467802721016


In [ ]:
#Generating time points to generate data from t -72 hours to t -24 hours, where t is TC formation/Non formation time. 

data2 = pd.read_csv(path1+BASIN+'OceanCenterCombined.csv', index_col=0)
sidIndex = data2.columns.get_loc('SID')
stimeIndex = data2.columns.get_loc('startTime')
catIndex = data2.columns.get_loc('cateogry')
latIndex = data2.columns.get_loc('lat')
lonIndex = data2.columns.get_loc('lon')

sids = pd.unique(data2['SID'])
alldata = []
values = data2.values
secondsIn24Hours = 24*60*60
secondsIn6Hours = 6*60*60
for val in values:
  sid = val[sidIndex]
  startTime = val[stimeIndex]
  cat = val[catIndex]
  lat = val[latIndex]
  lon = val[lonIndex]
  startDate =  datetime.datetime.strptime(startTime, '%Y-%m-%d %H:%M:%S')
  for i in range(8, -1, -1):
    data = []
    time1 = startDate - timedelta(seconds = secondsIn24Hours+ i*6*60*60)
    data.append(sid)
    data.append(time1.strftime('%Y-%m-%d %H:%M:%S'))
    data.append(lat)
    data.append(lon)
    data.append(cat)
    alldata.append(data)


data3 = pd.DataFrame(alldata, columns = ['SID', 'ISO_TIME','LAT','LON','cateogry'])
data3.to_csv(path1+BASIN+'OceanCenterCombined1.csv') 


In [6]:
import os
import os.path
#Function to check if a number if float or not
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

spread = 5 # Degree spread around center of cyclone

#Function to get u, v, z, r, t at three pressure levels  for a grid centerd at cyclone location. 
def getPressureData(cdsApi, year, month, date, time, lati, long, path):
  time1 = time.replace(':','_')
  fileName = path+ date +month+year+'_'+time1+'.grib'
  print('fileName  ', fileName)
  if os.path.isfile(fileName):
    print("file already exist")
    return
  cdsApi.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'geopotential', 'relative_humidity', 'temperature',
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '225', '500', '700',
        ],
        'year': [
            year,
        ],
        'month': [
            month,
        ],
        'day': [
            date,
        ],
        'time': [
            time,
        ],
        'format': 'grib',
        'area': [
            lati+spread, long-spread, lati-spread,
            long+spread,
        ],
    },
    fileName)
  
def createFolder(path):
  if not os.path.exists(path):
    os.makedirs(path)
    
#Rounding lat, long to the multiple of 0.25
def roundF(x, base=.25):
    return base * round(x/base)

In [7]:
# Downloading reanalysis data for a grid 5*5 degree at resolution 0.25  around TC
data2 = pd.read_csv(path1+BASIN+'OceanCenterCombined1.csv', index_col=0)
sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON')
count = 0
dataValues = data2.values
cdsApi = cdsapi.Client()
for data in dataValues:
  print('count  ', count)
  count = count + 1
  sid = data[sidIndex]
  #Creating folder for pressure data
  folder1 = path1+'data/pressure/'+sid+'/'
  createFolder(folder1)
  isoTime = data[isoTimeIndex]
  lat = data[latIndex]
  long = data[longIndex]
  date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')
  year = date.year

  year = str(year)
  month = date.month
  day = date.day
  time = date.hour

  if month < 10:
    month = '0'+str(month)
  else:
    month = str(month)

  if day < 10:
    day = '0'+str(day)
  else:
    day = str(day)

  if time<10:
    time = '0'+str(time)+':00'
  else:
    time = str(time) +':00'

  lat = roundF(lat)
  long = roundF(long)

  #print('day  ', day, ' month ', month, ' year ', year, ' time ', time, ' lat ', lat, ' long ', long)
  getPressureData(cdsApi, year, month, day, time, lat, long, folder1)

count   0
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/29082020_00_00.grib
file already exist
count   1
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/29082020_06_00.grib
file already exist
count   2
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/29082020_12_00.grib
file already exist
count   3
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/29082020_18_00.grib
file already exist
count   4
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/30082020_00_00.grib
file already exist
count   5
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/30082020_06_00.grib
file already exist
count   6
fileName   /content/drive/MyDrive/IJCAICodeToSubmit/WPSep2020ToSep2021/data/pressure/2020241N17131/300

In [ ]:
#Function that return the .grib file.
def getFileName(folder, isoTime):
    date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')

    year = date.year
    year = str(year)
    month = date.month
    day = date.day
    time = date.hour

    if month < 10:
      month = '0'+str(month)
    else:
      month = str(month)

    if day < 10:
      day = '0'+str(day)
    else:
      day = str(day)

    if time<10:
      time = '0'+str(time)+'_00'
    else:
      time = str(time) +'_00'

    file1 = folder+ day +month+year+'_'+time+'.grib'
    return file1

In [ ]:
pip install eccodes

In [ ]:
#Reading .grib files and extracting u, v, z fileds and SST for 33*33 grid.
#For each filed we have created a column for field values at each cell of grid. So for each fields we have 33*33 = 1089 columns.
import eccodes
from eccodes import *
data2 = pd.read_csv(path1+BASIN + 'OceanCenterCombined1.csv', index_col = 0, header=0)

sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON') 
catIndex = data2.columns.get_loc('cateogry') 

sids = pd.unique(data2['SID'])
gridSize = 41
alldata = []
colums = ['SID', 'ISO_TIME','LAT','LON']
check = True
count = -1
for sid in sids:
  count = count + 1
  if count < 1000:
    continue
  print('sid  ',sid, "count  ", count)
  cyclonData = data2[data2['SID'] == sid]
  valuesC = cyclonData.values
  length = len(valuesC)

  folder1 = path1 +'/data/pressure/'+sid+'/'
  for i in range(length):
    datac = []
    data = valuesC[i]
    isoTime = data[isoTimeIndex]
    lat = data[latIndex]
    long = data[longIndex]
    cate = data[catIndex]


    datac.append(sid)
    datac.append(isoTime)
    datac.append(lat)
    datac.append(long)

    pressureFile = getFileName(folder1, isoTime)
    
    print('pressureFile  ',pressureFile)
    
    id = codes_index_new_from_file(pressureFile, ["shortName",'level'])

    names = codes_index_get(id, "shortName")
    levels = codes_index_get(id, "level")
    
    #print("names  ", names)
    #print("levels  ",levels)

    for name in names:
      #Getting index for each name (u, v, z)
      codes_index_select(id, "shortName", name)
      for level in levels:
        #Getting index for each leval 225, 500, 700
        codes_index_select(id, "level", level)
        gid = codes_new_from_index(id)
        values = codes_get_values(gid)
        length = len(values)
        # Appening values of field for each location of grid. 
        for i in range(length):
            val = np.float32(values[i])
            datac.append(val)
            #Creating dataset columnds name
            if check:
                colums.append(name+str(level)+'_'+str(i))

    datac.append(cate)
    if check:
      colums.append('cateogry')
    alldata.append(datac)
    codes_index_release(id)
    check = False
  
dataa = pd.DataFrame(alldata, columns = colums)
dataa.to_csv(path1+BASIN+'reanalysisData.csv')